<h1 style="text-align:center">Decision Tree</h1>

[Tutorial](https://machinelearningcoban.com/tabml_book/ch_model/decision_tree.html?fbclid=IwAR1RTwuA9zgihGVN9Mam8E76Ha9OM7P49NhT-SDjcXB2qbDJAVazKO79j8U)

In [1]:
import pandas as pd
link = "https://media.githubusercontent.com/media/tiepvupsu/tabml_data/master/titanic/"
df = pd.read_csv(link + "train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
title_mapping = {"Mr" : 1, "Mrs": 2, "Miss": 3, "Rev": 4, "Master": 5}
sex_mapping = {"male" : 1, "female": 2}

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, load_iris

In [30]:
dataset = load_iris()
data = dataset.data
labels = dataset.target
len(data)

150

In [7]:
class Node:
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, label = None):
        """
        Node:
        Attribute
        Threshold
        left
        right
        label
        """
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        
        # Leaf Node
        self.label = label

In [32]:
class DecisionTree:
    def __init__(self, max_depth = 5, min_group = 2):
        self.root = None
        
        self.max_depth = max_depth
#       self.min_group = min_group
        
    def should_stop(self, datasets, depth):
        return datasets == None or depth >= self.max_depth
        
    def build_tree(self, datasets, depth):
        node_info = {}
        max_info_gain = - float("inf")
        
        if self.should_stop(datasets, depth): return None
        
        n_samples, n_features = datasets.shape
        n_features -= 1
        for feature_index in range(n_features):
            data_idx = datasets[:, feature_index]
            for threshold in data_idx:
                cur_data_left, cur_data_right = self.split_data(datasets, feature_index, threshold)
                if len(cur_data_left) > 0 and len(cur_data_right) > 0:
                    cur_info_gain = information_gain(datasets, cur_data_left, cur_data_right)
                    if cur_info_gain > max_info_gain:
                        max_info_gain = cur_info_gain
                        node_info["data_left"] = cur_data_left
                        node_info["data_right"] = cur_data_left
                        node_info["feature_index"] = feature_index
                        node_info["threshold"] = threshold

        node_left = build_tree(node_info["data_left"], depth + 1)
        node_right = build_tree(node_info["data_right"], depth + 1)
        root = Node(feature_index = node_info["feature_index"], 
                    threshold = node_info["threshold"], 
                    left = node_left, 
                    right = node_right)        
        
            
            
    
    def split_data(self, dataset, feature_index, threshold):
        data_left = np.array([row for row in dataset if dataset[feature_index] <= threshold])
        data_right = np.array([row for row in dataset if dataset[feature_index] > threshold])
        return data_left, data_right
    
    def information_gain(self, datasets, data_left, data_right ,mode = "gini"):
        count_ = len(datasets)
        count_left = len(data_left)
        count_right = len(data_right)
        weight = np.array([count_left, count_right]) / count_
        
        if mode == 'gini':
            root_info = self.gini(datasets)
            child_info = np.array([self.gini(data_left), self.gini(data_right)])
        else:
            root_info = self.entropy(datasets)
            child_info = np.array([self.entropy(data_left), self.entropy(data_right)])
            
        return root_info - weight.T @ child_info
            
    def entropy(self, datasets):
        _, counts = np.unique(datasets[:, -1], return_counts = True)
        total = counts.sum()
        prob = counts / total
        return - np.sum(prob * np.log2(prob), axis = 0)
    
    def gini(self, datasets):
        _, counts = np.unique(datasets[:, -1], return_counts = True)
        total = counts.sum()
        prob = counts / total
        return 1 - np.sum(prob ** 2, axis = 0)
        
    
    def fit(self):
        pass
    
    def predict(self):
        pass

    